In [6]:
!pip install tensorflow-addons

     -------------------------------------- 746.7/746.7 kB 4.7 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.optimizers import Adam

# 1. Pré-processar os dados
# Carregue seus datasets balanceados
train_data = pd.read_csv('../data/preprocessed/movies_genres_train_preprocessed.csv')
test_data = pd.read_csv('../data/preprocessed/movies_genres_test_preprocessed.csv')

category_columns = train_data.columns.drop(['Name', 'Description', 'Combined'])

# Defina os parâmetros de pré-processamento
max_features = 10000  # Número máximo de palavras a serem usadas (palavras mais frequentes)
maxlen = 200  # Número máximo de palavras no texto

# Tokenize os textos
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(train_data['Combined'])

X_train = tokenizer.texts_to_sequences(train_data['Combined'])
X_test = tokenizer.texts_to_sequences(test_data['Combined'])

# Padronize os textos
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

# Obtenha os labels one-hot encoded
y_train = train_data[category_columns].values
y_test = test_data[category_columns].values

# 2. Construir o modelo
embedding_dim = 128  # Dimensão do vetor de embedding
lstm_units = 64  # Unidades LSTM

model = Sequential([
    Embedding(max_features, embedding_dim, input_length=maxlen),
    Bidirectional(LSTM(lstm_units, return_sequences=True)),
    Bidirectional(LSTM(lstm_units)),
    Dense(len(category_columns), activation='sigmoid')
])

# 3. Treinar o modelo
optimizer = Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

batch_size = 32
epochs = 10

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

# 4. Avaliar o modelo
loss, accuracy = model.evaluate(X_test, y_test)
print('Loss:', loss)
print('Accuracy:', accuracy)

Epoch 1/10
150/150 [==============================] - 159s 780ms/step - loss: 0.2538 - accuracy: 0.2793 - val_loss: 0.2188 - val_accuracy: 0.2876
Epoch 2/10
150/150 [==============================] - 108s 721ms/step - loss: 0.2129 - accuracy: 0.2809 - val_loss: 0.2149 - val_accuracy: 0.2481
Epoch 3/10
150/150 [==============================] - 106s 704ms/step - loss: 0.2065 - accuracy: 0.2768 - val_loss: 0.2089 - val_accuracy: 0.2481
Epoch 4/10
150/150 [==============================] - 108s 718ms/step - loss: 0.1986 - accuracy: 0.3340 - val_loss: 0.2036 - val_accuracy: 0.3102
Epoch 5/10
150/150 [==============================] - 118s 786ms/step - loss: 0.1904 - accuracy: 0.3756 - val_loss: 0.2029 - val_accuracy: 0.3045
Epoch 6/10
150/150 [==============================] - 132s 879ms/step - loss: 0.1810 - accuracy: 0.4110 - val_loss: 0.1994 - val_accuracy: 0.3195
Epoch 7/10
150/150 [==============================] - 104s 695ms/step - loss: 0.1681 - accuracy: 0.4645 - val_loss: 0.1983 -

In [5]:
from sklearn.metrics import f1_score, average_precision_score, precision_recall_curve

y_pred = model.predict(X_test)
y_pred = np.round(y_pred)
f1_micro = f1_score(y_test, y_pred, average='micro')
f1_macro = f1_score(y_test, y_pred, average='macro')
precision_micro = precision_score(y_test, y_pred, average='micro')
precision_macro = precision_score(y_test, y_pred, average='macro')
recall_micro = recall_score(y_test, y_pred, average='micro')
recall_macro = recall_score(y_test, y_pred, average='macro')
print("F1-score (micro): {:.2f}".format(f1_micro))
print("F1-score (macro): {:.2f}".format(f1_macro))
print("Precision (micro): {:.2f}".format(precision_micro))
print("Precision (macro): {:.2f}".format(precision_macro))
print("Recall (micro): {:.2f}".format(recall_micro))
print("Recall (macro): {:.2f}".format(recall_macro))

f1_scores_per_class = f1_score(y_test, np.round(y_pred), average=None)
auc_pr_per_class = average_precision_score(y_test, y_pred, average=None)
for i, category in enumerate(category_columns):
    print("Class: {}".format(category))
    print("F1-score: {:.2f}".format(f1_scores_per_class[i]))
    print("AUC-PR: {:.2f}".format(auc_pr_per_class[i]))
    print("\n")

91/91 [==============================] - 22s 221ms/step
F1-score (micro): 0.41
F1-score (macro): 0.15
Precision (micro): 0.62
Precision (macro): 0.25
Recall (micro): 0.31
Recall (macro): 0.12


C:\Users\carol\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


NameError: name 'average_precision_score' is not defined